Bir Stacked AutoEncoder oluşturup kullanıcının bir filme kaç puan vereceğini tahmin edeceğiz.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# importing the dataset
# default seperatör ',' oluyor ama film isimleri virgünl içerebildiği için farklı seperator kullanıyoruz
# sütun başlıkları olmadığı için None, bazı film isimleri utf-8'de olmayan özel karakterler içeriyor
movies = pd.read_csv("data/BM/ml-1m/movies.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
users = pd.read_csv("data/BM/ml-1m/users.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
ratings = pd.read_csv("data/BM/ml-1m/ratings.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
print("movie\'s id, name, contents", len(movies), "\n", movies[:3], "\n")
print("user's id, sex, age, job, zip code", len(users), "\n", users[:3], "\n")
print("user's id, movie's id, ratings(1-5), timesteps", len(ratings), "\n", ratings[:3], "\n")

movie's id, name, contents 3883 
    0                        1                             2
0  1         Toy Story (1995)   Animation|Children's|Comedy
1  2           Jumanji (1995)  Adventure|Children's|Fantasy
2  3  Grumpier Old Men (1995)                Comedy|Romance 

user's id, sex, age, job, zip code 6040 
    0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117 

user's id, movie's id, ratings(1-5), timesteps 1000209 
    0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968 



In [3]:
# preparing train and testset
# bu klasörde 100k veri k=5 fold cros validation için 5e bölünmüş durumda base=train test=test, ama sadece birini kullanacağız
training_set = pd.read_csv("data/BM/ml-100k/u1.base", delimiter = "\t")
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv("data/BM/ml-100k/u1.test", delimiter = "\t")
test_set = np.array(test_set, dtype = "int")
print(len(training_set), len(test_set))
print(training_set[:3])
print(test_set[:3]) # yukarıdaki ratings'in aynısı

79999 19999
[[        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]]
[[        1        10         3 875693118]
 [        1        12         5 878542960]
 [        1        14         5 874965706]]


In [4]:
# Kişiler için matrisler oluşturacağız hangi tarz filmleri beğendiklerine yönelik, max id alsak users sayısını buluruz
nb_users = max(max(training_set[:, 0]), max(test_set[:, 0]))
nb_movies = max(max(training_set[:, 1]), max(test_set[:, 1]))
print(nb_users, nb_movies)

943 1682


In [5]:
# BM matris verisi istediği için users are rows & movies are columns olan matrixler oluşturuyoruz
def convert(data):
    new_data = list()
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users] # data[rows][condition], kullanıcının oyladığı filmleri aldık
        id_ratings = data[:, 2][data[:, 0] == id_users] # puanlamadığı filmler 0'lar ile doldurulacak
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings)) # torch expects list of lists
    return new_data

In [6]:
training_set = convert(training_set)
test_set = convert(test_set)
print(str(training_set[0])[:200])

[0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 1.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 5.0, 0.0, 2.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0,


In [7]:
# Converting the data into Torch tensors
# Tensorler(multi dimensional matrix with single type) npArray'lerden daha etkili ve hızlı işlem yapılabilmesini sağlıyorlar
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)
print(training_set[0])

tensor([0., 3., 4.,  ..., 0., 0., 0.])


### Architecture of the Neural Network

In [8]:
class SAE(nn.Module):
    
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20) # ilk hidden layerin 20 nodesi olacak, bu veri setine 20 uygunmuş
        self.fc2 = nn.Linear(20, 10) # ikinci saklı katmanda 10 node olacak, önceki katmanla full connection için 20 yazıldı
        self.fc3 = nn.Linear(10, 20) # encoder kısmı bitti decoder kısmınıda aynı yapalım
        self.fc4 = nn.Linear(20, nb_movies) # nasıl başladıysa öyle sonlansın simetrik bir ağ olsun
        self.activation = nn.Sigmoid() # başka aktivasyon fonksiyonları da seçilebilir
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [12]:
sae = SAE()
criterion = nn.MSELoss() # loss func
# başka optimizerlar da seçilebilir, lr=learningRate, weight_decat = reduces lr after each epoch and improve the model
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5) 

### Training the SAE

In [17]:
np_epoch = 200

for epoch in range(1, np_epoch + 1):
    train_loss = 0 # there is no loss yet
    s = 0. # number of users which didn't rate any movie
    
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0: # kullanıcı en az bir filmi puanlamışsa
            output = sae(input) # input = forward(x)'teki x oluyor
            target.require_grad = False
            output[target == 0] = 0 # hiç puanlamamış kullanıcılar kayda alınmayacak o yüzden sıfıra eşitleniyorlar
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) # be sure dominator is not zero (+ 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print("Epoch: " + str(epoch) + "Loss: " + str(train_loss/s))

Epoch: 1Loss: 1.7485072011386338
Epoch: 2Loss: 1.0961391585681826
Epoch: 3Loss: 1.0531720833133846
Epoch: 4Loss: 1.038160269860717
Epoch: 5Loss: 1.030888710324606
Epoch: 6Loss: 1.0264676410229745
Epoch: 7Loss: 1.0235527551521468
Epoch: 8Loss: 1.0220229222498596
Epoch: 9Loss: 1.0208120744092708
Epoch: 10Loss: 1.019512248354153
Epoch: 11Loss: 1.0187425976871562
Epoch: 12Loss: 1.0184008890246776
Epoch: 13Loss: 1.0178788119077617
Epoch: 14Loss: 1.0174358976060385
Epoch: 15Loss: 1.0172821987702128
Epoch: 16Loss: 1.0168427928875743
Epoch: 17Loss: 1.0168167989185848
Epoch: 18Loss: 1.0165568214256753
Epoch: 19Loss: 1.0163935127994115
Epoch: 20Loss: 1.0161844462347747
Epoch: 21Loss: 1.01609627164655
Epoch: 22Loss: 1.0158542801575348
Epoch: 23Loss: 1.0158134859802004
Epoch: 24Loss: 1.015778259771542
Epoch: 25Loss: 1.0155341732272432
Epoch: 26Loss: 1.015689520344161
Epoch: 27Loss: 1.0154221027242052
Epoch: 28Loss: 1.0152185463786298
Epoch: 29Loss: 1.0134677975765949
Epoch: 30Loss: 1.0112486894472

### Testing the SAE

In [20]:
test_loss = 0
s = 0.

for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.
print("Loss: " + str(test_loss/s))
# 5 yıldızlı puanlama sisteminde 0.94 1 yıldıza denk gelir. 4 yıldızlık bir film için +-1 hata payı vardır. 3,4,5 olabilir. 

Loss: 0.9461880169721086


sınıfa predict için eklenebilir<br>
<br>
def predict(self, x): # x: visible nodes<br>
        x = self.forward(x)<br>
        return x